In [1]:
import pandas as pd
import numpy as np 
import warnings
import sys
sys.path.append("../HistoricalData/")
from getData import get_data
import glob

warnings.filterwarnings('ignore')

from math import sin, cos, radians, pi
import time
from collections import defaultdict

import swifter

import dask.array as da
import dask.dataframe as dd

from tqdm import tqdm_notebook

In [2]:
MIN_LAT = 37.2781261
MAX_LAT = 38.063446
MIN_LON = -122.683496
MAX_LON = -121.814281


UP_LEFT = (38.063446, -122.683496)    
UP_RIGHT = (38.063446, -121.814281)   
DOWN_RIGHT = (37.2781261, -121.814281) 
DOWN_LEFT = (37.2781261, -122.683496)  

In [3]:
months  = pd.date_range(start = "2018/09/01", end = "2019/10/02", freq = 'MS') \
    .map(lambda x: x.strftime("%Y/%m/%d")).to_list()

In [4]:
month_ends = pd.date_range(start = "2018/09/01", end = "2019/11/02", freq = 'M') \
    .map(lambda x: x.strftime("%Y/%m/%d")).to_list()

In [6]:
START_HOUR = '0'
END_HOUR = '24'

tot_rows = 0

for START_DATE, END_DATE in zip(months, month_ends):
    df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, 
                  START_DATE, END_DATE, START_HOUR, END_HOUR, 'Monthly')
    
    tot_rows += df.shape[0]
    df['wind_direction'] = df['wind_direction'].map(lambda x: x if x is not None else 0)

    # fill in no wind times
    df['wind_speed'].fillna(0, inplace = True)
    df['wind_direction'].replace('', 0, inplace = True)

    # change VRB to 0
    vrb = df[df.wind_direction == "VRB"].index
    df.loc[vrb, 'wind_direction'] = 0
    df.loc[vrb, 'wind_speed'] = 0

    df['wind_x'] = df.apply(lambda x: x.wind_speed * cos(radians(int(x.wind_direction))), axis = 1)
    df['wind_y'] = df.apply(lambda x: x.wind_speed * sin(radians(int(x.wind_direction))), axis = 1)
    
    year,month,day = START_DATE.split("/")
    df.drop(columns=['a_h', 'day', 'daytype', 'device_loc_typ',
                   'hour', 'month', 'timeofday', 'uptime', 'wind_direction', 'wind_speed',
                    '0_3um', '0_5um', '10_0um', '1_0um',  '5_0um', 'agency_name',
                   'aqi',  'call_sign2', 'call_sign3', 'category', 'city',
                   'county', 'created',  'epa_pm25_unit', 
                   'full_aqs_code', 'gust_speed', 'gusts', 'hidden', 'high_reading_flag',
                   'interval', 'intl_aqs_code', 'is_owner','minute', 'parent_id', 'pm10_0', 'pm10_0_atm', 'pm1_0', 'pm1_0_atm',
                   'pm2_5_atm', 'pm2_5_cf_1', 'raw_concentration', 'report_modifier',
                   'rssi', 'sensor_name', 'site_name', 'sys_maint_reqd',
                    'thingspeak_primary_id',
                   'thingspeak_primary_id_read_key', 'thingspeak_secondary_id',
                   'thingspeak_secondary_id_read_key', 'variable_wind_info',
                   'variable_winds', 'wban_number', 'wind_data', 'year', 'zipcode',
                   'zulu_time', 'wkday', 'wind_compass']) \
        .to_parquet(f"{year}{month}_bigger.parquet", index = False, compression = 'gzip')

In [13]:
tot_rows

6145506

### making one big df

In [7]:
parq_files = sorted(glob.glob("*bigger.parquet"))

ddf = dd.read_parquet(parq_files.pop())

# remaining files
for pf in parq_files:
    ddf = ddf.append(dd.read_parquet(pf)).reset_index(drop = True)

In [9]:
ddf.to_parquet("big_combined.parquet", compression = 'gzip', write_index = False)

#### reloading it later

In [2]:
ddf = dd.read_parquet("big_combined.parquet")

### mapping stuff to a grid

In [3]:
boxes = pd.read_csv("../bigger_500m_grid.csv")

In [4]:
boxes.head()

,min_lat,max_lat,min_lon,max_lon,x,y,center_lat,center_lon,in_water,ndvi
0,37.278126,37.281700,-122.683496,-122.679004,0,0,37.279913,-122.68125,True,-2000
1,37.281700,37.285274,-122.683496,-122.679004,0,1,37.283487,-122.68125,True,-2000
2,37.285274,37.288847,-122.683496,-122.679004,0,2,37.287060,-122.68125,True,-2000
3,37.288847,37.292421,-122.683496,-122.679004,0,3,37.290634,-122.68125,True,-2000
4,37.292421,37.295994,-122.683496,-122.679004,0,4,37.294207,-122.68125,True,-1999


In [5]:
boxes[(boxes.in_water == False) & (boxes.ndvi >=0)].shape

(28095, 10)

In [6]:
boxes.x.max(), boxes.y.max()

(193, 220)

In [7]:
def get_coords(line):
    box = boxes[(boxes.min_lat < line.lat) & (boxes.max_lat > line.lat) & 
                (boxes.min_lon < line.lon) & (boxes.max_lon > line.lon)]
    assert box.shape[0] == 1
    return box.iloc[0, 4], box.iloc[0, 5] # x,y

slow af

#### option b

In [8]:
%%time
# option b
sensor_locs = ddf.compute()[['sensor_id', 'lat', 'lon']].drop_duplicates()
sensor_locs['xy_'] = sensor_locs.apply(get_coords, axis = 1)

CPU times: user 35.6 s, sys: 33.6 s, total: 1min 9s
Wall time: 12.5 s


In [9]:
# do all sensors have the same location all the time
sensor_locs['sensor_id'].astype(object).describe()

count       234
unique      227
top       37025
freq          3
Name: sensor_id, dtype: object

no

In [10]:
bad_ids = sensor_locs.groupby('sensor_id').count().reset_index().query('lat>1')['sensor_id']

In [11]:
sensor_locs[sensor_locs.sensor_id.isin(bad_ids)].sort_values(by=['sensor_id'])

,sensor_id,lat,lon,xy_
20176,18999,37.794788,-122.432496,"(55, 145)"
637211,18999,37.795219,-122.432641,"(55, 145)"
15767,23927,37.747370,-122.413213,"(60, 131)"
283184,23927,37.747370,-122.413213,"(60, 131)"
13238,36641,37.888464,-122.532814,"(33, 171)"
552957,36641,37.888413,-122.533124,"(33, 171)"
552996,37025,37.875881,-122.252691,"(95, 167)"
556680,37025,37.875936,-122.253167,"(95, 167)"
556803,37025,37.875921,-122.253082,"(95, 167)"
640520,38535,37.795382,-122.432451,"(55, 145)"


well at least they more or less stay in the same box

In [12]:
%%time
# option b, continued
sensor_locs = sensor_locs.drop_duplicates(subset=['sensor_id']) \
                         .assign(x = lambda d: sensor_locs['xy_'].map(lambda f:f[0]),
                                 y = lambda d: sensor_locs['xy_'].map(lambda f:f[1])) \
                         .set_index('sensor_id')

CPU times: user 5.12 ms, sys: 0 ns, total: 5.12 ms
Wall time: 4.33 ms


In [13]:
sensor_locs.xy_.nunique()

201

In [16]:
sensor_locs.head()

,lat,lon,xy_,x,y
sensor_id,,,,,
16939,37.722440,-122.439302,"(54, 124)",54,124
16919,37.722456,-122.439390,"(54, 124)",54,124
16931,37.722417,-122.439245,"(54, 124)",54,124
19173,37.722383,-122.439227,"(54, 124)",54,124
16947,37.722391,-122.439178,"(54, 124)",54,124


In [18]:
id_lookup = sensor_locs.T.to_dict()

In [15]:
ddf.head()

,2_5um,call_sign,created_at,epa_pm25_value,humidity,lat,lon,sensor_id,temperature,wind_x,wind_y
0,1.43,KSFO,2019/10/01T01:00,3.600000,53.0,37.72244,-122.439302,16939,66.0,-1.653273e-15,-9.0
1,1.76,KSFO,2019/10/01T01:10,3.633333,55.0,37.72244,-122.439302,16939,66.0,-1.653273e-15,-9.0
2,3.88,KSFO,2019/10/01T01:20,3.666667,56.0,37.72244,-122.439302,16939,65.0,-1.469576e-15,-8.0
3,4.85,KSFO,2019/10/01T01:30,3.700000,57.0,37.72244,-122.439302,16939,65.0,-1.469576e-15,-8.0
4,3.86,KSFO,2019/10/01T01:40,3.733333,58.0,37.72244,-122.439302,16939,65.0,-1.285879e-15,-7.0


In [19]:
ddf['x'] = ddf.apply(lambda s: id_lookup[s['sensor_id']]['x'], axis =1, meta = pd.Series())


In [20]:
ddf['y'] = ddf.apply(lambda s: id_lookup[s['sensor_id']]['y'], axis =1, meta = pd.Series())


### adding time_space_id

In [61]:
# are timestamps every 5 or 10 minutes
df['minute'].compute().unique()

array(['00', '10', '20', '30', '40', '50'], dtype=object)

In [21]:
## all this x and y shit is for faster neighbor lookup
import datetime

def time_space(line):
    """ takes x, y, created at
    
    returns string of form TTTT_x_y
    
    Where TTTT is unix timestamp divided by 600 (so an increment of 1 TTTT
    is equivalent to 10 minutes, or how often we get readings)
    
    """
    
    ts_ = int(datetime.datetime.strptime(line.created_at, "%Y/%m/%dT%H:%M").timestamp() / 600)
    return f"{ts_}_{line.x}_{line.y}"

In [22]:
%%time
ddf['time_space_id'] = ddf.apply(time_space, axis = 1, meta = pd.Series())

CPU times: user 6.46 ms, sys: 0 ns, total: 6.46 ms
Wall time: 5.98 ms


### adding satellite data

In [23]:
### adding satellite data
import rasterio
ndvi = rasterio.open("../nn/new_ndvi.tif") # normalized vegetation index
band_n = ndvi.read(1)

In [24]:
  
def get_ndvi(line):
    # get ndvi
    row, col = ndvi.index(line.lon, line.lat)
    
    return band_n[row,col]

In [25]:
boxes['ndvi'] = boxes.assign(lon = boxes['center_lon'], lat = boxes['center_lat']) \
                     .apply(get_ndvi, axis = 1)

In [26]:
boxes.head()

,min_lat,max_lat,min_lon,max_lon,x,y,center_lat,center_lon,in_water,ndvi
0,37.278126,37.281700,-122.683496,-122.679004,0,0,37.279913,-122.68125,True,-2000
1,37.281700,37.285274,-122.683496,-122.679004,0,1,37.283487,-122.68125,True,-2000
2,37.285274,37.288847,-122.683496,-122.679004,0,2,37.287060,-122.68125,True,-2000
3,37.288847,37.292421,-122.683496,-122.679004,0,3,37.290634,-122.68125,True,-2000
4,37.292421,37.295994,-122.683496,-122.679004,0,4,37.294207,-122.68125,True,-1999


In [27]:
sensor_locs['ndvi'] = sensor_locs.apply(get_ndvi, axis = 1)
# sensor_locs['image'] = sensor_locs.apply(get_sentinel, axis = 1)

In [28]:
# dataset_a.close()
# dataset_b.close()
ndvi.close()

In [29]:
sensor_locs.head()

,lat,lon,xy_,x,y,ndvi
sensor_id,,,,,,
16939,37.722440,-122.439302,"(54, 124)",54,124,3327
16919,37.722456,-122.439390,"(54, 124)",54,124,3327
16931,37.722417,-122.439245,"(54, 124)",54,124,3327
19173,37.722383,-122.439227,"(54, 124)",54,124,3327
16947,37.722391,-122.439178,"(54, 124)",54,124,3327


In [30]:
sensor_locs.to_csv("sensor_locs_big_box.csv")

In [31]:
sensor_locs['xy_'].nunique()

201

In [32]:
ssd = sensor_locs.T.to_dict()

### final prep before splitting

In [62]:
ddf.columns

Index(['2_5um', 'call_sign', 'created_at', 'epa_pm25_value', 'humidity', 'lat',
       'lon', 'sensor_id', 'temperature', 'wind_x', 'wind_y', 'x', 'y',
       'time_space_id'],
      dtype='object')

In [33]:
# get vegetation index
ddf['ndvi'] = ddf.apply(lambda x: sensor_locs.loc[x.sensor_id, 'ndvi'], axis = 1, meta = pd.Series())

In [34]:
ddf['ts_'] = ddf \
    .apply(lambda x: int(datetime.datetime.strptime(x.created_at, "%Y/%m/%dT%H:%M").timestamp()),
           axis = 1, meta = pd.Series())

In [37]:
import modelUtils as mu
avg_hum_temp = ddf.groupby('ts_').agg({'humidity':'mean', 
                                             'temperature':'mean',
                                             'epa_pm25_value':'mean'})

In [38]:
%%time


ddf['humidity'] = ddf \
    .apply(lambda l: mu.fill_in_avgs(l, 'humidity',avg_hum_temp), axis = 1, meta = pd.Series())
ddf['temperature'] = ddf \
    .apply(lambda l: mu.fill_in_avgs(l, 'temperature',avg_hum_temp), axis = 1, meta = pd.Series())
ddf['epa_pm25_value'] = ddf \
    .apply(lambda l: mu.fill_in_avgs(l, 'epa_pm25_value',avg_hum_temp), axis = 1, meta = pd.Series())

CPU times: user 96 ms, sys: 0 ns, total: 96 ms
Wall time: 94 ms


In [3]:
#ddf.to_parquet("big_combined.parquet", compression = 'gzip', write_index = False)

In [40]:
#from sklearn.model_selection import train_test_split
from dask_ml.model_selection import train_test_split
target_var = '2_5um'

X_train_and_dev, X_test, y_train_and_dev, y_test = train_test_split(ddf, 
                                                                    ddf[target_var], 
                                                                    test_size=0.20, 
                                                                    random_state=42)

X_train, X_dev, y_train, y_dev = train_test_split(X_train_and_dev, y_train_and_dev, test_size=0.125, random_state=42)

### now adding neighbors in time and space

In [41]:
%%time
# create a key value mapping of the from time_space_id:(row number in X_train)
neighbor_lookup = defaultdict(list)
_ = X_train.apply(lambda x: neighbor_lookup[x.time_space_id].append(x.name), axis = 1)

CPU times: user 8.11 ms, sys: 0 ns, total: 8.11 ms
Wall time: 7.61 ms


In [6]:
X_train.head()

,2_5um,epa_pm25_value,humidity,lat,lon,sensor_id,temperature,wind_x,wind_y,x,y,time_space_id,ndvi,ts_,is_dense
0,4.00,5.800000,49.0,37.787307,-122.417252,4770,73.0,-2.736161e+00,-7.517541,26,24,2559606_26_24,931,1535763600,False
1,4.95,6.566667,51.0,37.787307,-122.417252,4770,72.0,-1.389185e+00,-7.878462,26,24,2559607_26_24,931,1535764200,False
2,1.41,7.333333,53.0,37.787307,-122.417252,4770,71.0,-1.562834e+00,-8.863270,26,24,2559608_26_24,931,1535764800,False
3,3.71,8.100000,54.0,37.787307,-122.417252,4770,70.0,-1.653273e-15,-9.000000,26,24,2559609_26_24,931,1535765400,False
10,4.80,10.266667,59.0,37.787307,-122.417252,4770,68.0,0.000000e+00,0.000000,26,24,2559616_26_24,931,1535769600,False


In [42]:
def get_neighbors_space_time(line, train_df):
    """
    Inputs: single observation, a training dataframe, and a time delta
    Outputs: vector of length 25 corresponding to surrounding neighbor observations
    """
    
    t, x, y = line.time_space_id.split("_")

    t = int(t)
    x = int(x)
    y = int(y)
    neighbors = np.zeros((25))
    
    c = 0
    for i in range(-2,3):
        for j in range(-2,3):
            #if i == 0 and j == 0:
            n = neighbor_lookup[f"{t}_{x+i}_{y+j}"] # get rows in train_df for that particular time-block
            
            if n:
                tmp_df = train_df.loc[n, ['sensor_id','2_5um']]
                neighbors[c] = tmp_df[tmp_df.sensor_id != line.sensor_id]['2_5um'].mean()
                
            c += 1
    
    return neighbors

In [45]:
%%time

X_train_neighbors = X_train \
    .apply(lambda line: get_neighbors_space_time(line, X_train), axis =1, meta=pd.Series())




CPU times: user 34.1 ms, sys: 3.31 ms, total: 37.4 ms
Wall time: 36.1 ms


In [63]:
X_train_neighbors.compute().shape

(4303186,)

In [ ]:
X_train_neighbors.compute().mean(axis = 1)

In [46]:
%%time
X_dev_neighbors = X_dev \
    .apply(lambda x: get_neighbors_space_time(x, X_train), axis =1, meta = pd.Series())

CPU times: user 33.4 ms, sys: 2.18 ms, total: 35.6 ms
Wall time: 34.2 ms


In [51]:
train_cut = (y_train < 300)
dev_cut = (y_dev < 300)

In [48]:
baseline_features = ['epa_pm25_value', 'humidity', 'temperature', 'wind_x', 'wind_y', 'ndvi']
# no elevation

# cuts not yet applied
X_train_base = X_train[baseline_features].values
X_dev_base = X_dev[baseline_features].values

In [61]:
%%time
train = da.concatenate((X_train_base.compute_chunk_sizes(), 
                        da.stack(X_train_neighbors.values.compute_chunk_sizes())), axis = 1)

KeyboardInterrupt: 

In [ ]:
dev = da.concatenate((X_dev_base.compute_chunk_sizes(), 
                      da.stack(X_dev_neighbors.values.compute_chunk_sizes())), axis = 1)

In [59]:
type(X_train_neighbors)

dask.dataframe.core.Series

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt

def eval_model(actuals, preds, actuals_train, preds_train):
    print("Train MSE:", mean_squared_error(actuals_train, preds_train))
    print("Train MAE:", mean_absolute_error(actuals_train, preds_train))
    print("Train R2:", r2_score(actuals_train, preds_train))
    
    print("Dev MSE:", mean_squared_error(actuals, preds))
    print("Dev MAE:", mean_absolute_error(actuals, preds))
    print("Dev R2:", r2_score(actuals, preds))
    
    errors = actuals - preds
    
    f, a = plt.subplots(ncols = 3, figsize = (15,5))
    a[0].scatter(actuals, preds)
    a[0].set_xlabel("actual values")
    a[0].set_ylabel("predicted dev values")
    
    a[1].scatter(actuals, errors)
    a[1].set_xlabel("actual values")
    a[1].set_ylabel("errors")
    
    a[2].scatter(preds, errors)
    a[2].set_xlabel("predicted values")
    a[2].set_ylabel("errors")

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score